<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_1_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    982      0 --:--:-- --:--:-- --:--:--   982
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'rt',
 'to',
 'the',
 'for',
 'a',
 'in',
 'you',
 'is',
 'of',
 'on',
 'disease',
 'and',
 'amp',
 'i',
 'gluten',
 'with',
 'free',
 'your',
 'it',
 'my',
 'have',
 'this',
 'new',
 'are',
 'glutenfree',
 'about',
 'our',
 'what',
 'do',
 'nt',
 'celiac',
 'from',
 'be',
 'out',
 'now',
 'not',
 'at',
 'or',
 'that',
 'how',
 'we',
 'can',
 'via',
 'food',
 'its',
 'right',
 'tip',
 'more',
 'use',
 'if',
 'all',
 'dblclick',
 'pic',
 'products',
 'w',
 'diet',
 'as',
 'up',
 'why',
 'so',
 'know',
 'me',
 'check',
 'review',
 'here',
 'love',
 'by',
 'get',
 'just',
 're',
 'but',
 'thanks',
 'm',
 'help',
 'an',
 'will',
 'great',
 'gfb',
 'was',
 'could',
 'posted',
 'recipe',
 'has',
 'post',
 'who',
 's',
 'may',
 '30',
 'us',
 'read',
 'people',
 'there',
 'heres',
 'some',
 'jennieo',
 'need',
 'like',
 'living',
 'life',
 'when',
 'eat',
 'make',
 'archives',
 'going',
 '\r \n ',
 'test

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.139118,4.241713,0.269547,04:06


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.074307,3.705239,0.330835,04:12


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.785021,3.666664,0.336193,04:07


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.714876,3.629497,0.341135,04:06


In [0]:
learn.save_encoder('bigtweet_fold1_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.601663,4.529374,0.288136,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.833107,4.192050,0.329298,01:30


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.503391,3.976228,0.355932,03:41


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.293521,3.780540,0.384988,03:31
1,3.137682,3.734802,0.392252,03:27


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.128669,3.686373,0.394673,03:36
1,2.959857,3.633602,0.406780,03:26


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.896296,3.580793,0.414044,03:20
1,2.799138,3.552833,0.412833,03:42


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.791886,3.514551,0.417676,03:35
1,2.656996,3.468764,0.428571,03:40


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.727213,3.496241,0.414044,03:33
1,2.599253,3.491321,0.420097,03:31


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.620848,3.533680,0.424939,03:22
1,2.622791,3.452481,0.418886,03:40
2,2.545168,3.424966,0.435835,03:37
3,2.434854,3.448447,0.428571,03:21


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.431194,3.428502,0.430993,03:24
1,2.475072,3.456085,0.416465,03:38
2,2.357647,3.417597,0.433414,03:26
3,2.255889,3.390867,0.433414,03:18


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.358052,3.474247,0.432203,03:36
1,2.427808,3.504308,0.429782,03:26
2,2.400331,3.507604,0.437046,03:29
3,2.267765,3.459139,0.434625,03:37


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.382861,3.628445,0.427361,03:33
1,2.375291,3.579543,0.432203,03:29
2,2.350704,3.576517,0.429782,03:40
3,2.207048,3.470406,0.433414,03:39


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.244913,3.536361,0.423729,03:31
1,2.317462,3.578649,0.426150,03:27
2,2.274169,3.561989,0.440678,03:27
3,2.195014,3.537924,0.445521,03:34


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.229974,3.619839,0.440678,03:40
1,2.324970,3.590588,0.439467,03:27
2,2.205102,3.574836,0.437046,03:26
3,2.201726,3.612016,0.437046,03:27


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.230084,3.667997,0.427361,03:20
1,2.270186,3.602359,0.427361,03:30
2,2.186936,3.521254,0.434625,03:29
3,2.147562,3.603482,0.439467,03:38


In [0]:
# save the best model

learn.save_encoder('ask-1-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[111, 111, 1026, 1026, 1026, 1026, 1026, 359, 20, 896, 214, 775, 27, 414, 896, 210, 864, 864, 864, 864, 775, 741, 289, 289, 37, 864, 1035, 1035, 655, 183, 502, 49, 896, 214, 214, 214, 214, 325, 53, 53, 53, 864, 896, 61, 62, 896, 70, 794, 1035, 896, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 686, 896, 896, 896, 896, 896, 289, 896, 289, 80, 783, 80, 80, 80, 80, 491, 864, 896, 89, 214, 1026, 1026, 1026, 94, 94, 94, 94, 427, 94, 896, 896, 896, 814, 228, 228, 775, 1035, 28, 765, 298, 298, 775, 100, 100, 419, 774, 775, 765, 298, 896, 896, 1035, 289, 1035, 182, 864, 190, 289, 462, 182, 864, 864, 864, 864, 289, 1035, 896, 117, 89, 125, 125, 125, 630, 630, 125, 125, 125, 125, 125, 125, 125, 125, 125, 303, 289, 606, 591, 265, 135, 94, 139, 896, 265, 142, 995, 289, 864, 854, 265, 896, 896, 896, 896, 111, 289, 211, 910, 289, 896, 896, 896, 228, 246, 289, 228, 82, 182, 1035, 1035, 864, 864, 289,

In [44]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

178
0.20530565167243367
